In [1]:
from IPython.display import display, HTML 
display(HTML("""
<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:15pt;}
div.output {font-size:15pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:15pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:15px;}
</style>
 """))

<span style="color:red; font-size: 35px; font-weight: bold;"> ch14.웹데이터 수집 </span></br>
# 1절. BeautifulSoup과 parser

pip install bs4 아나콘다를 설치하면 7500개여개의 패키지 설치

- 공식 사이트 : https://www.crummy.com/software/BeautifulSoup/
- Docs : https://www.crummy.com/software/BeautifulSoup/bs4/doc/

In [1]:
import requests #HTTP 요청 처리 lib
from requests_file import FileAdapter

In [15]:
s = requests.Session() #HTTP 요청관리 위한 세션 객체
s.mount("file://", FileAdapter())
response = s.get('file:///ai_x/lecNote/01_python/data/ch14_sample.html')
response

<Response [200]>

In [16]:
response.status_code #상태 코드  
#200 : 정상 상태
#404 : 없는 페이지
#406 : get,post 오류
#500 : 파이썬 문법 오류 (내가 만든 사이트)

200

In [17]:
response.content #바이너리 형식의 내용

b'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n  <meta charset="UTF-8">\r\n</head>\r\n<body>\r\n  <h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>\r\n  <h1 class="css">Hi, CSS</h1>\r\n  <div id="subject">subject \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90 \xec\x95\x88\xec\x9d\x98 \xeb\x82\xb4\xec\x9a\xa9</div>\r\n  <p>CSS \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90\xeb\x8a\x94 \xeb\x8b\xa4\xec\x96\x91\xed\x95\x9c \xea\xb3\xb3\xec\x97\x90\xec\x84\x9c \xed\x99\x9c\xec\x9a\xa9\xeb\x90\xa9\xeb\x8b\x88\xeb\x8b\xa4</p>\r\n  <div class="contents">\r\n    \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90\xeb\xa5\xbc \xec\x96\xb4\xeb\x96\xbb\xea\xb2\x8c \xec\x9e\x91\xec\x84\xb1\xed\x95\x98\xeb\x8a\x90\xeb\x83\x90\xec\x97\x90 \xeb\x94\xb0\xeb\x9d\xbc\r\n    <span>\xeb\x8b\xa4\xeb\xa5\xb8<b>\xec\x9a\x94\xec\x86\x8c\xea\xb0\x80 \xeb\xb0\x98\xed\x99\x98</b></span>\xeb\x90\xa9\xeb\x8b\x88\xeb\x8b\xa4\r\n  </div>\r\n  <div>CSS \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90\xeb\x8a\x94 \xeb\x8b\xa4\xec\x96\x91\xed\

In [6]:
response.content.decode('utf-8')

'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n  <meta charset="UTF-8">\r\n</head>\r\n<body>\r\n  <h1 class="greeting css" id="text">Hello, CSS</h1>\r\n  <h1 class="css">Hi, CSS</h1>\r\n  <div id="subject">subject 선택자 안의 내용</div>\r\n  <p>CSS 선택자는 다양한 곳에서 활용됩니다</p>\r\n  <div class="contents">\r\n    선택자를 어떻게 작성하느냐에 따라\r\n    <span>다른<b>요소가 반환</b></span>됩니다\r\n  </div>\r\n  <div>CSS 선택자는 다양한 곳에 <b>활용</b>됩니다</div>\r\n</body>\r\n</html>'

In [7]:
response.text

'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n  <meta charset="UTF-8">\r\n</head>\r\n<body>\r\n  <h1 class="greeting css" id="text">Hello, CSS</h1>\r\n  <h1 class="css">Hi, CSS</h1>\r\n  <div id="subject">subject 선택자 안의 내용</div>\r\n  <p>CSS 선택자는 다양한 곳에서 활용됩니다</p>\r\n  <div class="contents">\r\n    선택자를 어떻게 작성하느냐에 따라\r\n    <span>다른<b>요소가 반환</b></span>됩니다\r\n  </div>\r\n  <div>CSS 선택자는 다양한 곳에 <b>활용</b>됩니다</div>\r\n</body>\r\n</html>'

In [18]:
#html 파싱
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content, #response.text 
                    "html.parser") #soup객체 만들기
# soup 웨크롤링 할 수 있게 보여줌

In [25]:
# 1.soup.select_one('선택자') : 해당 선택자 처음 하나만
el = soup.select_one('h1') #처음 나오는 h1태그 하나만
print('el :', el)
print('el.text :', el.text)
print('el.string :', el.string)
print('el의 속성추가 :', el.attrs) #딕셔너리로 나타냄
print('el의 title속성 :', el.attrs['title']) #title의 속성만 가져오기
print('el의 title속성 :', el.attrs.get('title')) #get함수가 오류가 안 나서 좋음
print('el의 이름 :', el.name)

el : <h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>
el.text : Hello, CSS
el.string : Hello, CSS
el의 속성추가 : {'class': ['greeting', 'css'], 'id': 'text', 'title': 'greeting'}
el의 title속성 : greeting
el의 title속성 : greeting
el의 이름 : h1


In [33]:
# 2.soup.select('선택자') : 해당 선택자 모든 엘리먼트를 리스트로 나타냄
el = soup.select('h1')
print('리스트 el :', el)
print('el의 text들 :' ,[e.text for e in el])
print('el의 속성들 :', [e.attrs for e in el])
print('el의 class 속성들 :', [e.attrs.get('class') for e in el])

리스트 el : [<h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
el의 text들 : ['Hello, CSS', 'Hi, CSS']
el의 속성들 : [{'class': ['greeting', 'css'], 'id': 'text', 'title': 'greeting'}, {'class': ['css']}]
el의 class 속성들 : [['greeting', 'css'], ['css']]


In [41]:
#select_one('속성자')와 find(태그,속성) 비교
print('select_one :',soup.select_one('h1.css'))
print('find :', soup.find('h1',{'class':'css'}))
print('find :', soup.find('h1', class_='greeting'))
print()
print('select_one :', soup.select_one('h1#text'))
print('find :', soup.find('h1', {'id':'text'}))
print()
print('select')

select_one : <h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>
find : <h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>
find : <h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>

select_one : <h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>
find : <h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>

select


In [44]:
# select('선택자')와 find_all(태그,속성) 비교
print('모든 h1.css, span 태그(select) :', 
      soup.select('h1.css, span'))
print('모든 h1.css, span 태그(find_all) :',
     soup.find_all(['h1','span'],class_='css')) #span은 안 들어온거 아닌지?
print() #개행

모든 h1.css, span 태그(select) : [<h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>, <span>다른<b>요소가 반환</b></span>]
모든 h1.css, span 태그(find_all) : [<h1 class="greeting css" id="text" title="greeting">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]


In [49]:
# 없는 엘리먼트 찾기  (a태그)
print('find_all :', soup.find_all('a', class_='css')) #없으면 빈리스트로
print('find :', soup.find('a',class_='css')) #없으면 none으로
print('select :', soup.select('a.css')) #빈 리스트로
print('select_one :', soup.select_one('a.css')) #없으면 none으로

find_all : []
find : None
select : []
select_one : None


# 2절. 정적 웹 데이터 수집(정적 웹크롤링)
- json,html, xml
## 2.1 JSON파일
- request모둘(get)
- urllib.request모듈(urlopen)

In [ ]:
# 크롤링 허용 범위는 사이트마다 다름 / robots.txt에서 확인할 수 있습니다.

In [133]:
# 방법 1. json
import requests
response = requests.get('http://api.github.com')
response, response.status_code

(<Response [403]>, 403)

In [132]:
# 방법 2. json
from urllib.request import urlopen
response = urlopen('http://api.gitgub.com')
response

URLError: <urlopen error [Errno 11001] getaddrinfo failed>

In [135]:
text = '{"속성1":"값1", "속성2":"값2"}'
type(text)

str

In [136]:
# 문자(딕셔너리 타입)를 딕셔너리
# '{"속성1":"값1", "속성2":"값2"}' => {"속성1":"값1", "속성2":"값2"}
import json 
json.loads(text)

{'속성1': '값1', '속성2': '값2'}

## 2.2html 파일
### 1)환율정보 가져오기(네이버->증권->시장지표)
https://finance.naver.com/marketindex/

In [76]:
# 방법 1. requests
import requests 
from bs4 import BeautifulSoup
url = 'https://finance.naver.com/marketindex/'
response = requests.get(url)
# response, response.status_code
# response.content
# response.content.decode('cp949') == response.text
soup = BeautifulSoup(response.text,
                    'html.parser')

In [81]:
# 방법 2. urlopen
from urllib.request import urlopen
response = urlopen(url)
# response.status 상태 코드 (정상상태 확인)
# response.read() #content와 동일한 값 출력
# response.read().decode('cp949')
soup = BeautifulSoup(response,'html.parser') #text 사용 불가 

In [99]:
title = soup.select('h3.h_lst > span.blind')
for t in title:
    print(t.text, end='\n')

미국 USD
일본 JPY(100엔)
유럽연합 EUR
중국 CNY
달러/일본 엔
유로/달러
영국 파운드/달러
달러인덱스
WTI
휘발유
국제 금
국내 금


In [107]:
price = soup.select('div.head_info > span.value')
[p.text for p in price]
[round(float(p.text.replace(',','')))for p in price]
[round(float(''.join(p.text.split(',')))) for p in price]

[1375, 953, 1555, 191, 144, 1, 1, 99, 61, 1633, 3300, 146173]

In [96]:
#,기준으로 빈스트링으로 연결 join
out = '1,374.70'
round(float(''.join(out.split(','))))

1375

In [113]:
unit = soup.select('div.head_info > span > span.blind')
unit = [u.text for u in unit]
unit.insert(7,'') #7번째 index에 '' 추가
unit

['원', '원', '원', '원', '엔', '달러', '달러', '', '달러', '원', '달러', '원']

In [115]:
status = soup.select('div.head_info > span.blind')
[s.text for s in status]
# status

['하락', '하락', '하락', '하락', '상승', '하락', '하락', '상승', '하락', '하락', '하락', '상승']

In [116]:
[t.text for t in title]
[p.text for p in price]
unit
[s.string for s in status]

['하락', '하락', '하락', '하락', '상승', '하락', '하락', '상승', '하락', '하락', '하락', '상승']

In [117]:
len(title), len(price), len(unit), len(status)

(12, 12, 12, 12)

In [123]:
for idx in range(len(title)):
    print("{}.{} : {}{}-{}".format(idx+1,
                                    title[idx].text,
                                    price[idx].text,
                                    unit[idx],
                                    status[idx].text))

1.미국 USD : 1,375.00원-하락
2.일본 JPY(100엔) : 952.51원-하락
3.유럽연합 EUR : 1,555.06원-하락
4.중국 CNY : 191.06원-하락
5.달러/일본 엔 : 144.3800엔-상승
6.유로/달러 : 1.1342달러-하락
7.영국 파운드/달러 : 1.3520달러-하락
8.달러인덱스 : 99.4200-상승
9.WTI : 60.89달러-하락
10.휘발유 : 1633.17원-하락
11.국제 금 : 3300.4달러-하락
12.국내 금 : 146173.4원-상승


In [126]:
for t,p,u,s in zip(title,price,unit,status):
    print("{} : {}{}-{}".format(t.text,
                                p.text,
                                u,
                                s.text))

미국 USD : 1,375.00원-하락
일본 JPY(100엔) : 952.51원-하락
유럽연합 EUR : 1,555.06원-하락
중국 CNY : 191.06원-하락
달러/일본 엔 : 144.3800엔-상승
유로/달러 : 1.1342달러-하락
영국 파운드/달러 : 1.3520달러-하락
달러인덱스 : 99.4200-상승
WTI : 60.89달러-하락
휘발유 : 1633.17원-하락
국제 금 : 3300.4달러-하락
국내 금 : 146173.4원-상승


In [128]:
for idx,(t,p,u,s) in enumerate(zip(title,price,unit,status)):
    print("{}.{} : {}{}-{}".format(idx+1,
                                t.text,
                                p.text,
                                u,
                                s.text))

1.미국 USD : 1,375.00원-하락
2.일본 JPY(100엔) : 952.51원-하락
3.유럽연합 EUR : 1,555.06원-하락
4.중국 CNY : 191.06원-하락
5.달러/일본 엔 : 144.3800엔-상승
6.유로/달러 : 1.1342달러-하락
7.영국 파운드/달러 : 1.3520달러-하락
8.달러인덱스 : 99.4200-상승
9.WTI : 60.89달러-하락
10.휘발유 : 1633.17원-하락
11.국제 금 : 3300.4달러-하락
12.국내 금 : 146173.4원-상승


### 2) 이번주 로또번호 출력 
- google에 "로또번호 당첨번호" 검색
https://dhlottery.co.kr/gameResult.do?method=byWin


In [140]:
#방법 1의 soup객체 생성
import requests
from bs4 import BeautifulSoup
url = 'https://dhlottery.co.kr/gameResult.do?method=byWin'
response = requests.get(url)
soup = BeautifulSoup(response.text,'html.parser')
# response.status_code

200

```
1173회 (2025년 05년 24일 추첨)
당첨번호 [1 5 18 20 30 35]
보너스 3
```

In [261]:
time = soup.select_one('div.win_result > h4 > strong').text
date = soup.select_one('div.win_result > p.desc').text
title1 = soup.select_one('div.num.win > strong').text
win = soup.select('div.num.win > p> span')
winnum = [int(w.text) for w in win]
bonusnum = soup.select_one('div.num.bonus > p > span').text
title2 = soup.select_one('div.num.bonus > strong').text
print(time,date)
print(title1, ":", winnum)
print(title2, ":", [int(bonusnum)])

1173회 (2025년 05월 24일 추첨)
당첨번호 : [1, 5, 18, 20, 30, 35]
보너스 : [3]


In [210]:
num = soup.select('div.win_result > h4 > strong')
[n.text for n in num]
# num

['1173회']

In [198]:
date = soup.select('div.win_result > p')
[d.text for d in date]

['(2025년 05월 24일 추첨)']

In [199]:
title = soup.select('div.num.win > strong')
[t.text for t in title]

['당첨번호']

In [205]:
winnum = soup.select('div.num.win > P')
[w.text for w in winnum]

['\n1\n5\n18\n20\n30\n35\n']

In [206]:
bonusnum = soup.select('div.num.bonus > p')
[b.text for b in bonusnum]

['3']

In [256]:
#방법 2의 soup객체 생성
from urllib.request import urlopen
response = urlopen(url)
soup = BeautifulSoup(response,'html.parser')

In [302]:
win_result = soup.find('div', class_='win_result')
times = win_result.find('strong').text
date = soup.find('p', class_='desc').text
print(times, date)
num_win = soup.find('div',class_=['win'])
title1 = num_win.find('strong').text
lotto_number = num_win.find_all('span') #배열
print(title1,[int(lotto.text) for lotto in lotto_number])
title2 = num_bonus.find('strong').text
num_bonus = soup.find('div', class_=['bonus'])
bonus_num = num_bonus.find_all('span')
print(title2,[int(bonus.text) for bonus in bonus_num])

1173회 (2025년 05월 24일 추첨)
당첨번호 [1, 5, 18, 20, 30, 35]
보너스 [3]


### 3) 다음 검색 리스트
https://search.daum.net/search?w=tot&DA=YZR&t__nil_searchbox=btn&q=%EC%95%BC%EA%B5%AC+
```
no    title    link
0    [비즈 나우] 비트코인               https://v.daum.net/v/20250528075215864
1    [비트코인 2025] 백악관 크립토 차르  https://v.daum.net/v/20250528103907230
```

In [321]:
keyword = '비트코인'
url = 'https://search.daum.net/search?nil_suggest=btn&w=news&DA=SBC&cluster=y&q='\
        +keyword
print(url)

https://search.daum.net/search?nil_suggest=btn&w=news&DA=SBC&cluster=y&q=비트코인


In [333]:
# 웹크롤링 방법1 (리스트안에 딕셔너리로 만들기)
import requests
from bs4 import BeautifulSoup
keyword = '비트코인'
url = 'https://search.daum.net/search?nil_suggest=btn&w=news&DA=SBC&cluster=y&q='\
        +keyword
response = requests.get(url)
soup = BeautifulSoup(response.text,
                    'html.parser')
items_find_list=[] #검색할 결과를 담을 dict list
items_el = soup.select('div.item-title > strong.tit-g.clamp-g > a')
len(items_el)
for idx, item in enumerate(items_el):
    items_find_list.append({'no': idx,
                            'title' : item.text,
                            'link' : item.attrs.get('href')})
import pandas as pd
pd.DataFrame(items_find_list)

,no,title,link
0,0,[비즈 나우] 비트코인 2025 컨퍼런스 개막…'전략자산' 선언 코앞,http://v.daum.net/v/20250528075215864
1,1,[비트코인 2025] 백악관 크립토 차르 “美 정부 비트코인 추가 매입 검토…부채...,http://v.daum.net/v/20250528103907230
2,2,"비트코인, 트럼프 미디어 비축 소식에도 주춤…1억5100만원대",http://v.daum.net/v/20250528095112897
3,3,“맥O날드보다 맛없어!” 혹평이 가득한 트럼프 만찬과 비트코인 피자데이[엠블록레터],http://v.daum.net/v/20250528143002735
4,4,"美 상원의원 ""트럼프 대통령, 비트코인법 지지""",http://v.daum.net/v/20250528090342853
5,5,[비트코인 2025] 로빈후드 창업자 “토큰화 증권은 美 ‘자본 패권’ 키우는 수단”,http://v.daum.net/v/20250528110600761
6,6,"'비트코인 빚투' 스트레티지, 또 샀다…보유량 58만개 돌파",http://v.daum.net/v/20250528042404292
7,7,"""비트코인 산다""…트럼프家, 25억달러 자금 조달 추진[코인브리핑]",http://v.daum.net/v/20250528111837527
8,8,"숨 고르는 비트코인, 10만8000달러선 '주춤'",http://v.daum.net/v/20250528095921258
9,9,"트럼프미디어그룹, 25억 달러 규모 자금 조달 통해 비트코인 매입 예고",http://v.daum.net/v/20250528084245020


In [344]:
# 리스트안에 리스트 만들기
import requests
from bs4 import BeautifulSoup
keyword = '비트코인'
url = 'https://search.daum.net/search?nil_suggest=btn&w=news&DA=SBC&cluster=y&q='\
        +keyword
response = requests.get(url)
soup = BeautifulSoup(response.text,
                    'html.parser')
items_find_list=[] #검색한 결과를 담을 2차원 리스트
items_el = soup.select('div.item-title > strong.tit-g.clamp-g > a')
for idx, item in enumerate(items_el):
    items_find_list.append([idx, item.text, item.attrs.get('href')])

# items_find_list
import pandas as pd
pd.DataFrame(items_find_list, columns=['no','title','link'])

,no,title,link
0,0,[비트코인 2025] 백악관 크립토 차르 “美 정부 비트코인 추가 매입 검토…부채...,http://v.daum.net/v/20250528103907230
1,1,[비즈 나우] 비트코인 2025 컨퍼런스 개막…'전략자산' 선언 코앞,http://v.daum.net/v/20250528075215864
2,2,“맥O날드보다 맛없어!” 혹평이 가득한 트럼프 만찬과 비트코인 피자데이[엠블록레터],http://v.daum.net/v/20250528143002735
3,3,"블랙록, 자사 비트코인 ETF 보유량 25% 확대…기관 투자 본격화 신호탄",http://v.daum.net/v/20250528151802080
4,4,"비트코인, 트럼프 미디어 비축 소식에도 주춤…1억5100만원대",http://v.daum.net/v/20250528095112897
5,5,"'비트코인 빚투' 스트레티지, 또 샀다…보유량 58만개 돌파",http://v.daum.net/v/20250528042404292
6,6,"美 상원의원 ""트럼프 대통령, 비트코인법 지지""",http://v.daum.net/v/20250528090342853
7,7,"숨 고르는 비트코인, 10만8000달러선 '주춤'",http://v.daum.net/v/20250528095921258
8,8,[비트코인 2025] 로빈후드 창업자 “토큰화 증권은 美 ‘자본 패권’ 키우는 수단”,http://v.daum.net/v/20250528110600761
9,9,"트럼프미디어그룹, 25억 달러 규모 자금 조달 통해 비트코인 매입 예고",http://v.daum.net/v/20250528084245020


In [358]:
# 다음 뉴스 검색(키워드,원하는 페이지수)
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import time
keyword = '비트코인'
page = 3
# url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=NTB&enc=utf8&cluster=y&q={keyword}&p={page}'
# response = requests.get(url)

url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=NTB&enc=utf8&cluster=y'
params = {'q':keyword, 'p':page}
response = requests.get(url,params=params)
soup = BeautifulSoup(response.text,
                     'html.parser')

items_find_list=[] #검색할 결과를 담을 dict list
items_el = soup.select('div.item-title > strong.tit-g.clamp-g > a')
len(items_el)
for idx, item in enumerate(items_el):
    items_find_list.append({'no': idx,
                            'title' : item.text,
                            'link' : item.attrs.get('href')})
    print({'no':(page-1)*10+ idx,
                            'title' : item.text,
                            'link' : item.attrs.get('href')})

{'no': 20, 'title': ' [영상] “강세장 진입한 비트코인, 내년 2분기까지 최소 20만~30만 달러 갈 듯” ', 'link': 'http://v.daum.net/v/20250527090137566'}
{'no': 21, 'title': ' 전력거래소, `공감·혁신` 토대 조직문화 바꾸기 ', 'link': 'http://v.daum.net/v/20250528152647580'}
{'no': 22, 'title': ' “미국 대통령이 이래도 되나”…트럼프 일가, 4조원 유치해 대놓고 코인 투자 ', 'link': 'http://v.daum.net/v/20250527220300328'}
{'no': 23, 'title': ' 트럼프미디어, 비트코인 3.4조 투자 발표에 주가 10% 폭락 ', 'link': 'http://v.daum.net/v/20250528080206055'}
{'no': 24, 'title': ' 트럼프 미디어, 비트코인 사나…가상자산 매입 위해 30억 달러 조달 추진 ', 'link': 'http://v.daum.net/v/20250527043000368'}
{'no': 25, 'title': ' 전력거래소, 2025년 공감과 혁신 특강 개최 ', 'link': 'http://v.daum.net/v/20250528150120264'}
{'no': 26, 'title': ' <단독>"예배 참석하면 코인 드려요. 5년 안에 비트코인 넘어섭니다"..신앙심 노린 코인 유사 사기 기승 ', 'link': 'http://v.daum.net/v/20250528083323774'}
{'no': 27, 'title': ' 뉴욕증시 휴장에 10.9만弗선 숨고른 비트코인 [매일코인] ', 'link': 'http://v.daum.net/v/20250527103810840'}
{'no': 28, 'title': ' 이재명 공약집…권력기관 고강도 개혁·AI 통한 성장에 방점 ', 'link': 'http://v.daum.net/v/202505281

In [ ]:
# 함수 만들기 

In [3]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import time
def collect_list(keyword,page=1):
    'keyword로 다음 검색한 결과(해당 page)를 return'
    url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=NTB&enc=utf8&cluster=y'
    params = {'q':keyword, 'p':page}
    response = requests.get(url,params=params)
    soup = BeautifulSoup(response.text,
                         'html.parser')
    items_find_list=[] #검색할 결과를 담을 dict list
    items_el = soup.select('div.item-title > strong.tit-g.clamp-g > a')
    for idx, item in enumerate(items_el):
        items_find_list.append({'no':(page-1)*10+ idx,
                                'title' : item.text,
                                'link' : item.attrs.get('href')})
    return items_find_list

In [366]:
collect_list('빨래',4)

[{'no': 30,
  'title': ' 어쩌면 마지막 빨래 ',
  'link': 'http://v.daum.net/v/20250421143123301'},
 {'no': 31,
  'title': ' ‘관세전쟁’에 빨래 횟수도 줄이는 미국인들 ',
  'link': 'http://v.daum.net/v/20250425123746580'},
 {'no': 32,
  'title': ' 밥 짓기·설거지·빨래 강요, 괴롭힘 신고도 묵살···새마을금고 간부들 1700만원 배상 ',
  'link': 'http://v.daum.net/v/20250428152014225'},
 {'no': 33,
  'title': ' 빨래 횟수도 줄이는 미국인들…실적 전망 줄줄이 낮춘다 ',
  'link': 'http://v.daum.net/v/20250425111350870'},
 {'no': 34,
  'title': ' 빨래건조대 혁신… 모터로 건조시간 35%↓ ',
  'link': 'http://v.daum.net/v/20250409012011659'},
 {'no': 35,
  'title': " 문앞에 놔두면 빨래 끝…세탁도 '앱 클릭' ",
  'link': 'http://v.daum.net/v/20250513160655495'},
 {'no': 36,
  'title': ' 박승희, 시모가 곱게 갠 빨래→아수라장 옷방에 철푸덕 (동상이몽2)[결정적장면] ',
  'link': 'http://v.daum.net/v/20250422060825114'},
 {'no': 37,
  'title': ' ‘제이쓴♥’ 홍현희, 60억 압구정 아파트 나와 집 없이 떠돌아다녀 “벌써 10일째” ',
  'link': 'http://v.daum.net/v/20250528145050679'},
 {'no': 38,
  'title': ' 하원미 "하숙집 엄마처럼 빨래, 밥, 청소"…추신수 월봉 70만원 시절에도 살뜰 내조(\'추신수몰래\') ',
  'link': 'http://

In [4]:
#원하는 keyword로 다음검색 (append X)
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import time

result = []
keyword = '키위'
pages = 4
for page in range(1,pages+1):
    result.extend(collect_list(keyword,page))
    time.sleep(3)
result_df = pd.DataFrame(result)
result_df.head(3)


,no,title,link
0,0,‘배부른 영양실조’에 빠진 한국인… 하루 키위 한 알이 해답?,http://v.daum.net/v/20250528175139032
1,1,‘저속노화’ 정희원 “매일 키위 2개 껍질째 먹어요”,http://v.daum.net/v/20250528172706940
2,2,"""키위 1알로 비타민 부족 개선""…제스프리, '영양소 밀도 식단' 발표",http://v.daum.net/v/20250528103005788


In [6]:
keywords=['키위','복숭아']
pages = 2
result0 = [] #키위 검색결과 50개
result1 = [] #복숭아 검색결과 50개
for i,keyword in enumerate(keywords):
    for page in range(1, pages+1):
        print(f'~~~~{i}번째 {keyword} {page} 검색중입니다~~~~~')
        if i ==0:
            result0.extend(collect_list(keyword,page))
        else:
            result1.extend(collect_list(keyword,page))
        time.sleep(4)

~~~~0번째 키위 1 검색중입니다~~~~~
~~~~0번째 키위 2 검색중입니다~~~~~
~~~~1번째 복숭아 1 검색중입니다~~~~~
~~~~1번째 복숭아 2 검색중입니다~~~~~


In [7]:
result0_df = pd.DataFrame(result0)
result0_df.head()

,no,title,link
0,0,‘배부른 영양실조’에 빠진 한국인… 하루 키위 한 알이 해답?,http://v.daum.net/v/20250528175139032
1,1,‘저속노화’ 정희원 “매일 키위 2개 껍질째 먹어요”,http://v.daum.net/v/20250528172706940
2,2,"""키위 1알로 비타민 부족 개선""…제스프리, '영양소 밀도 식단' 발표",http://v.daum.net/v/20250528103005788
3,3,"""저는 키위를 껍질째 먹어요""…제스프리, '영양소 밀도 식단' 연구 발표",http://v.daum.net/v/20250528133336313
4,4,"""영양 불균형 해소"" 제스프리가 소개하는 키위의 영양학적 가치는",http://v.daum.net/v/20250528153045857


### 4)User-agent를 추가하여 크롤링
- urlopen()함수를 사용하면 크롤링이 안 되는 사이트
- User-agent를 추가하여 크롤링(브라우저)

In [340]:
# 웹크롤링 방법2
from urllib.request import urlopen
import urllib.parse

word = '비트코인'
word = urllib.parse.quote(word)
# print(word)
url = 'https://search.daum.net/search?nil_suggest=btn&w=news&DA=SBC&cluster=y&q='+word
response = urlopen(url)
soup = BeautifulSoup(response,
                    'html.parser')

items_find_list=[] #검색할 결과를 담을 dict list
items_el = soup.select('div.item-title > strong.tit-g.clamp-g > a')
# len(items_el)
for idx, item in enumerate(items_el):
    items_find_list.append({'no': idx,
                            'title' : item.text,
                            'link' : item.attrs.get('href')})
import pandas as pd
pd.DataFrame(items_find_list)


""


꼭 User-Agent를 사용하여야 하는 경우 : https://www.melon.com/chart/index.htm

In [31]:
# 방법 1
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
url='https://www.melon.com/chart/'
# melonpage = requests.get(url)
# melonpage.status_code #오류

406

In [29]:
# 방법2.
from urllib.request import urlopen, Request
# import urllib.parse
# melonpage = urlopen(url) 에러남
# melonpage = requests.get(url)
# melonpage.status_code


In [35]:
#User-Agent 추가
headers = {'user-agent':
           'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36'}
#방법2
request = Request(url,headers=headers)
melonpage = urlopen(request)

#방법1
melonpage = requests.get(url, headers=headers)
soup = BeautifulSoup(melonpage.text, #melonpage.content
                    'html.parser')
# soup

In [84]:
import requests
from bs4 import BeautifulSoup
url='https://www.melon.com/chart/'
# 멜론 순위, 노래제목, 가수 , 좋아요갯수 
# 1위 | 너에게 닿기를 | 10CM | ♥ 85,756 
title = soup.select('div.ellipsis.rank01 > span')
title
artist = soup.select('div.ellipsis.rank02 > span.checkEllipsis')
artist
len([t.text.strip()for t in title])
len([art.text for art in artist])
for idx,(t,art) in enumerate(zip(title,artist)):
    print('{}순위 | 노래 제목:{} | 가수:{}'.format(idx+1,
                                          t.text.strip(),
                                          art.text))
# [t.text for t in title]
# artist = soup.select_one('div.ellipsis.rank02 > span.checkEllipsis').text
# artist

# print(title,artist)

1순위 | 노래 제목:너에게 닿기를 | 가수:10CM
2순위 | 노래 제목:Never Ending Story | 가수:아이유
3순위 | 노래 제목:Drowning | 가수:WOODZ
4순위 | 노래 제목:like JENNIE | 가수:제니 (JENNIE)
5순위 | 노래 제목:모르시나요(PROD.로코베리) | 가수:조째즈
6순위 | 노래 제목:THUNDER | 가수:세븐틴 (SEVENTEEN)
7순위 | 노래 제목:TOO BAD (feat. Anderson .Paak) | 가수:G-DRAGON
8순위 | 노래 제목:네모의 꿈 | 가수:아이유
9순위 | 노래 제목:Whiplash | 가수:aespa
10순위 | 노래 제목:HOME SWEET HOME (feat. 태양, 대성) | 가수:G-DRAGON
11순위 | 노래 제목:나는 반딧불 | 가수:황가람
12순위 | 노래 제목:어제보다 슬픈 오늘 | 가수:우디 (Woody)
13순위 | 노래 제목:오늘만 I LOVE YOU | 가수:BOYNEXTDOOR
14순위 | 노래 제목:REBEL HEART | 가수:IVE (아이브)
15순위 | 노래 제목:HAPPY | 가수:DAY6 (데이식스)
16순위 | 노래 제목:빨간 운동화 | 가수:아이유
17순위 | 노래 제목:Flower | 가수:오반(OVAN)
18순위 | 노래 제목:APT. | 가수:로제 (ROSÉ), Bruno Mars
19순위 | 노래 제목:청춘만화 | 가수:이무진
20순위 | 노래 제목:HOT | 가수:LE SSERAFIM (르세라핌)
21순위 | 노래 제목:10월 4일 | 가수:아이유
22순위 | 노래 제목:한 페이지가 될 수 있게 | 가수:DAY6 (데이식스)
23순위 | 노래 제목:Last Scene (Feat. 원슈타인) | 가수:아이유
24순위 | 노래 제목:HBD | 가수:세븐틴 (SEVENTEEN)
25순위 | 노래 제목:MY LOVE(2025) | 가수:이예은, 아샤트리, 전건호
26순위 | 노래 제목:그날이 오면 | 가수:투모로우바이투게더

### 5)네이버 지식인으로 검색(open API 사용X)
- 특정 keyword를 특정페이지 수(3)만큼 검색한 결과를 데이터프레임으로 출력

In [6]:
# 방법1
from requests import get
from bs4 import BeautifulSoup
keyword = '챗지피티' 
url = 'https://kin.naver.com/search/list.naver?query={}'.format(keyword)
print(url)
# response = get(url)
# response.status_code #url 잘 들어왔는지 확인 필요

soup =BeautifulSoup(response.text,
                   'html.parser')
# soup

https://kin.naver.com/search/list.naver?query=챗지피티


In [11]:
# 방법2
from urllib.request import urlopen
from urllib.parse import quote
# 챗지피티 -> %EC%B1%97%EC%A7%80%ED%94%BC%ED%8B%B0 #(url인코딩된 문자열)
keyword = quote(keyword)  # url 인코딩된 문자열로 전환
url = 'https://kin.naver.com/search/list.naver?query={}'.format(keyword)
# response = urlopen(url) #에러발생
print(url)
response = urlopen(url)
soup = BeautifulSoup(response, #response.reade()
                    'html.parser')

https://kin.naver.com/search/list.naver?query=%25252525EC%25252525B1%2525252597%25252525EC%25252525A7%2525252580%25252525ED%2525252594%25252525BC%25252525ED%252525258B%25252525B0


In [32]:
#페이징 포함(select)(keysord를 pages수 만큼 검색한 결과를 데이터프레임에)
import requests
from bs4 import BeautifulSoup
import pandas as pd

keyword = '쳇지피티'
pages = 3
items_list = [] #크롤링 한 데이터를 담을 list
for page in range(1,pages+1):
    url = f'https://kin.naver.com/search/list.naver?query={keyword}&page={page}'
    # print(url)
    # 해당 url의 soup객체 -> 글제목, link를 가져와서 items_list에 append
    response = requests.get(url)
    # print(response.status_code) 정상 접근 가능한지 여부 확인
    soup = BeautifulSoup(response.text, 'html.parser')
    items = soup.select('dt > a')
    #print([item.text for item in items])
    # print([item.attrs['href'] for item in items]) #link가져오기
    for item in items:
        items_list.append([item.text, item.attrs.get('href')])
# print(len(items_list), '개 데이터')
# print(items_list[:2])
df = pd.DataFrame(items_list, columns=['title','link'])
df.head()

30 개 데이터
[['쳇지피티 논문 출처', 'https://kin.naver.com/qna/detail.naver?d1id=4&dirId=40608&docId=471249146&qb=7LOH7KeA7ZS87Yuw&enc=utf8'], ['쳇지피티 한도', 'https://kin.naver.com/qna/detail.naver?d1id=1&dirId=1060101&docId=484917198&qb=7LOH7KeA7ZS87Yuw&enc=utf8']]


,title,link
0,쳇지피티 논문 출처,https://kin.naver.com/qna/detail.naver?d1id=4&...
1,쳇지피티 한도,https://kin.naver.com/qna/detail.naver?d1id=1&...
2,쳇지피티 사용 확인 AI,https://kin.naver.com/qna/detail.naver?d1id=5&...
3,쳇지피티 이거 뭐예요?해킹 뭐그런거...,https://kin.naver.com/qna/detail.naver?d1id=1&...
4,쳇지피티 ai 소라 질문이요,https://kin.naver.com/qna/detail.naver?d1id=1&...


In [35]:
#페이징 포함 (find_all)(keysord를 pages수 만큼 검색한 결과를 데이터프레임에)
import requests
from bs4 import BeautifulSoup
import pandas as pd

keyword = '쳇지피티'
pages = 3
items_list = [] #크롤링 한 데이터를 담을 list
for page in range(1,pages+1):
    url = f'https://kin.naver.com/search/list.naver?query={keyword}&page={page}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    dts = soup.find_all('dt') #dt안에 a태그가 있음
    for dt in dts:
        item = dt.find('a') #dt안에 a태그 가져오기
        items_list.append([item.text, item.attrs.get('href')])
df = pd.DataFrame(items_list, columns=['title','link'])
df.head()

,title,link
0,쳇지피티 논문 출처,https://kin.naver.com/qna/detail.naver?d1id=4&...
1,쳇지피티 한도,https://kin.naver.com/qna/detail.naver?d1id=1&...
2,쳇지피티 사용 확인 AI,https://kin.naver.com/qna/detail.naver?d1id=5&...
3,쳇지피티 이거 뭐예요?해킹 뭐그런거...,https://kin.naver.com/qna/detail.naver?d1id=1&...
4,쳇지피티 ai 소라 질문이요,https://kin.naver.com/qna/detail.naver?d1id=1&...


In [39]:
#함수 만들기 (keysord를 pages수 만큼 검색한 결과를 데이터프레임에)
def get_items_data(keyword,pages):
    import requests
    from bs4 import BeautifulSoup
    import pandas as pd
    items_list = [] #크롤링 한 데이터를 담을 list
    for page in range(1,pages+1):
        url = f'https://kin.naver.com/search/list.naver?query={keyword}&page={page}'
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        dts = soup.find_all('dt') #dt안에 a태그가 있음
        for dt in dts:
            item = dt.find('a') #dt안에 a태그 가져오기
            items_list.append([item.text, item.attrs.get('href')])
    return pd.DataFrame(items_list, columns=['title','link']).head(2)

In [42]:
get_items_data('참새',2)

,title,link
0,죽은 참새 시체를 발로 건드렸는데,https://kin.naver.com/qna/detail.naver?d1id=7&...
1,맨손으로 참새 잡았을 때 조류인플루엔자...,https://kin.naver.com/qna/detail.naver?d1id=7&...


### 6)네이버 지식인으로 검색 (open API 사용O)
- 특정 keyword를 특정 데이터 수(30)만큼 검색한 결과를 데이터프레임으로 출력
    - .env에 발급받은 key를 저장 -> load (pip install python-dotenv)

In [44]:
# 환경변수 가져오기
from dotenv import load_dotenv
import os 
load_dotenv()#dotenv_path='.env' 기본값 이므로 생략
# print(os.getenv('Client_ID'))
# print(os.getenv('Client_Secret'))

True

In [52]:
# 네이버 검색 API 예제 - 지식in 검색(방법2)
import os
import sys
import urllib.request
client_id = os.getenv('Client_ID')
client_secret = os.getenv('Client_Secret')
encText = urllib.parse.quote("복숭아")
url = "https://openapi.naver.com/v1/search/kin.json?query=" + encText # JSON 결과
# url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # XML 결과

headers = {'X-Naver-Client-Id' : client_id,
           'X-Naver-Client-Secret' : client_secret}
request = urllib.request.Request(url, headers=headers)

# request = urllib.request.Request(url)
# request.add_header("X-Naver-Client-Id",client_id)
# request.add_header("X-Naver-Client-Secret",client_secret)

response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(type(response_body.decode('utf-8'))) #문자 str 
    import json
    data = json.loads(response_body.decode('utf-8')) #json형태의 str을 딕셔너리 데이터로 변환
    print(type(data))
#     print(response_body.decode('utf-8')) #여기중에 items만 필요한걸 확인
    print(data['items'])
else:
    print("Error Code:" + rescode)


<class 'str'>
<class 'dict'>
[{'title': '<b>복숭아</b>가 너무 먹고싶어요...', 'link': 'https://kin.naver.com/qna/detail.naver?d1id=7&dirId=70101&docId=484591750&qb=67O17Iit7JWE&enc=utf8', 'description': '<b>복숭아</b> 알러지가 있는 사람입니다 원래부터 <b>복숭아</b> 알러지가 있던 건 아닌데 고3때 갑자기 체질이 바뀌면서 <b>복숭아</b> 알러지가 생겼고 그때부터 <b>복숭아</b>가 들어간 건 아무것도 못 먹었어요 그래도 그냥저냥 잘 살아왔는데... '}, {'title': '사랑니 발치 후 <b>복숭아</b> 먹어도 되나요?', 'link': 'https://kin.naver.com/qna/detail.naver?d1id=7&dirId=70205&docId=474408092&qb=67O17Iit7JWE&enc=utf8', 'description': '... 말랑한 <b>복숭아</b>가 너무 먹고싶네요..ㅜㅜ 사랑니 발치 후 말랑한 <b>복숭아</b> 먹어도 될까요?? 그리고 죽을 먹으려고... 사랑니 발치 후 <b>복숭아</b>와 죽에 대해 질문 주셨군요. 지금 상태가 많이 불편하지는 않으신가요? ㅜㅠ 첫째, <b>복숭아</b>는... '}, {'title': '<b>복숭아</b> 알러지 새로살구', 'link': 'https://kin.naver.com/qna/detail.naver?d1id=7&dirId=70301&docId=480105278&qb=67O17Iit7JWE&enc=utf8', 'description': '살구가 <b>복숭아</b>인가요? <b>복숭아</b>향이라길래 <b>복숭아</b>알러지인데 새로살구 먹어도되나요? 안녕하세요. 대한한의사협회... 살구와 <b>복숭아</b>는 다른 과일입니다. 살구에 대한 알러지가 딱히 없다면 드셔도 무방할 것 같습니다. 조금 드셔 보셔서... '}, {'

In [83]:
# keyword를 display수만큼 지식in 검색결과를 데이터프레임에 넣기
import requests
import pandas as pd
import json
from dotenv import load_dotenv
import os
load_dotenv()
client_id = os.getenv('Client_ID')
client_secret = os.getenv('Client_Secret')
keyword = '쳇지피티'
cnt = 30
url = f'https://openapi.naver.com/v1/search/kin.json?query={keyword}&display={cnt}'
headers = {'X-Naver-Client-Id' : client_id,
           'X-Naver-Client-Secret' : client_secret}
response = requests.get(url,headers=headers)
# response.status_code
# items = json.loads(response.text['items']) #:방법1과 방법2
items = response.json()['items'] #방법1에서만 가능한 함수
items_list = [] # items를 담을것
# print(items[0])
for item in items:
    items_list.append([item.get('title').replace('<b>','').replace('</b>',''), #item['title']로 써도 됨
                       item.get('link'),
                       item.get('description').replace('<b>','').replace('</b>','')
                      ])
# items_list[:2]
df = pd.DataFrame(items_list, columns=['title','link','description'])
df

,title,link,description
0,쳇지피티 논문 출처,https://kin.naver.com/qna/detail.naver?d1id=4&...,"쳇지피티한테 어떤 내용에 관한 논문을 요구하고, 해당... 지피티가 스스로 여러 논..."
1,쳇지피티 한도,https://kin.naver.com/qna/detail.naver?d1id=1&...,쳇지피티 무료로 쓰고 있는데 쳇지피티로 소설 하나 쓰고 있거든요 근데 한도가 초과됐...
2,쳇지피티 사용 확인 AI,https://kin.naver.com/qna/detail.naver?d1id=5&...,쳇지피티 사용 여부 확인하는 무료 AI 뭐가 있나요?? 좀 정확하고 확실한거로 알려...
3,쳇지피티 이거 뭐예요?해킹 뭐그런거 당하는거 아니죠?,https://kin.naver.com/qna/detail.naver?d1id=1&...,아니 제가 크롬을 쓰는데 자꾸 쳇지피티가 한글번역도 이상하게 되고 말도 끊기는 거예...
4,쳇지피티 ai 소라 질문이요,https://kin.naver.com/qna/detail.naver?d1id=1&...,쳇지피티 플러스 이용중이구요 어제까지만 해도 왼쪽 메뉴에 '소라' 가 있어서요 해서...
5,쳇지피티 초기화,https://kin.naver.com/qna/detail.naver?d1id=1&...,쳇지피티 대화 목록이 사라진건 아니고 맥락에 안 맞게 말하길래 한도 초과... 챗 ...
6,학교 생기부 쳇지피티,https://kin.naver.com/qna/detail.naver?d1id=4&...,학교에서 생기부를 작성할 때 쳇지피티를 그대로 배낀 보고서를 내면 대학에서 잡는다는...
7,제가 직접 쓴 자소서 쳇지피티,https://kin.naver.com/qna/detail.naver?d1id=4&...,제가 직접 제 이야기로 자소서를 쓰고 쳇지피티한테 보내고 지피티가 수정해준 내용 참...
8,쳇지피티 질문입니다.,https://kin.naver.com/qna/detail.naver?d1id=1&...,요즘 쳇지피티가 유행이길래 한번 깔아봤는데 인사조차 못알아먹는데 어떻게 하나요..?...
9,쳇지피티 채팅 문제,https://kin.naver.com/qna/detail.naver?d1id=1&...,쳇지피티3.0 무료버전을 종종 사용합니다 제가 질문을 &quot;이번에 전시회를 할...


In [84]:
# 함수 작성
def get_naver_kin(keyword,cnt):
    import requests
    import pandas as pd
    import json
    from dotenv import load_dotenv
    import os
    load_dotenv()
    client_id = os.getenv('Client_ID')
    client_secret = os.getenv('Client_Secret')
    url = f'https://openapi.naver.com/v1/search/kin.json?query={keyword}&display={cnt}'
    headers = {'X-Naver-Client-Id' : client_id,
               'X-Naver-Client-Secret' : client_secret}
    response = requests.get(url,headers=headers)
    # response.status_code
    # items = json.loads(response.text['items']) #:방법1과 방법2
    items = response.json()['items'] #방법1에서만 가능한 함수
    items_list = []8
    # items
    for item in items:
        items_list.append([item.get('title').replace('<b>','').replace('</b>',''), #item['title']로 써도 됨
                           item.get('link'),
                           item.get('description')
                          ])
    # items_list[:2]
    return pd.DataFrame(items_list, columns=['title','link','description'])

In [85]:
get_naver_kin('복숭아',3)

,title,link,description
0,복숭아가 너무 먹고싶어요...,https://kin.naver.com/qna/detail.naver?d1id=7&...,<b>복숭아</b> 알러지가 있는 사람입니다 원래부터 <b>복숭아</b> 알러지가 ...
1,사랑니 발치 후 복숭아 먹어도 되나요?,https://kin.naver.com/qna/detail.naver?d1id=7&...,... 말랑한 <b>복숭아</b>가 너무 먹고싶네요..ㅜㅜ 사랑니 발치 후 말랑한 ...
2,복숭아 알러지 새로살구,https://kin.naver.com/qna/detail.naver?d1id=7&...,살구가 <b>복숭아</b>인가요? <b>복숭아</b>향이라길래 <b>복숭아</b>알...


### qize) 네이버 open API를 이용해서 청바지 이미지 100건 데이터를 csv 파일로 저장
    데이터 : 제목, 이미지링크, 썸네일링크, 높이, 폭

In [9]:
# def get_naver_img(keyword):
import requests
import pandas as pd
import json
from dotenv import load_dotenv
import os
load_dotenv()
keyword = '청바지'
client_id = os.getenv('Client_ID')
client_secret = os.getenv('Client_Secret')
url = f'https://openapi.naver.com/v1/search/image?query={keyword}&display=100'
headers = {'X-Naver-Client-Id' : client_id,
           'X-Naver-Client-Secret' : client_secret}
response = requests.get(url, headers=headers)
items = json.loads(response.text)['items']
items_list  = [] #items_list초기화 
for idx, item in enumerate(items):
    title = item.get('title')
    link = item.get('link')
    thumbnail = item.get('thumbnail')
    sizeheight = item.get('sizeheight')
    sizewidth = item.get('sizewidth')
    
    items_list.append([
        idx+1,
        title,
        link,
        thumbnail,
        sizeheight,
        sizewidth])

    #link와 thumbnal 파일 저장
df = pd.DataFrame(items_list, columns=['no','tilte','link','thumbnail','sizeheight','sizewidth'])
df.to_csv(f'data/ch14_{keyword}.csv', index=False)
df.head()

,no,tilte,link,thumbnail,sizeheight,sizewidth
0,1,BINIWU 심플 빈티지 워싱 청바지 남자 봄가을 루즈핏 통바지 28 빈티지블루,http://shopping.phinf.naver.net/main_5172227/5...,https://search.pstatic.net/common/?type=b150&s...,800,800
1,2,중년 청바지 아이스 팬츠 남성 얇은 슬랙스 바지남 긴바지,http://shopping.phinf.naver.net/main_5314890/5...,https://search.pstatic.net/common/?type=b150&s...,1000,1000
2,3,사계절 남자 부드러운 스판 청바지 34 72 5 77 5kg,http://shopping.phinf.naver.net/main_5452984/5...,https://search.pstatic.net/common/?type=b150&s...,800,800
3,4,가우리 봄 가을 프리미엄 중년 남성 청바지 CRO 스판 청 팬츠 - 청바지 | 쿠팡,https://thumbnail8.coupangcdn.com/thumbnails/r...,https://search.pstatic.net/sunny/?type=b150&sr...,492,492
4,5,청바지 남자 여름 얇은 레져 텐셀 아메리칸 빈티지 청바지 남자 카우형 남자 스타일 팬츠,http://shopping.phinf.naver.net/main_5188539/5...,https://search.pstatic.net/common/?type=b150&s...,600,600


In [10]:
df.loc[0,'link']

'http://shopping.phinf.naver.net/main_5172227/51722270863.20241203000649.jpg'

In [11]:
df[df['link'].str.find('?')!=-1]

,no,tilte,link,thumbnail,sizeheight,sizewidth
55,56,텐바이텐 10X10 : 여성 겨울청바지 20대 깔끔일자핏기모청바지 하이웨이스트 30대,http://thumbnail.10x10.co.kr/webimage/image/ba...,https://search.pstatic.net/sunny/?type=b150&sr...,500,500


In [139]:
df[~df['link'].str.endswith('jpg')].loc[5,'link']

'http://shop1.phinf.naver.net/20250503_295/1746250611381Vinxl_JPEG/21192829047529977_2039968073.jpeg'

In [165]:
def save_img(attr, idx, query, url):
    import requests
    '{attr}_{idx}_{query}.{확장자} 이미지 저장'
    file_extension = url.split('.')[-1]
    quote_index = file_extension.find('?')
    
    if quote_index != -1: #-1이 아니라면
        file_extension = file_extension[:quote_index]
#         if len(file_extension) > 10:
            
#     print(file_extension)
    img = requests.get(url).content #바이널 파일
    open(f'ch14_image/{attr}_{idx}_{query}.{file_extension}', 'wb').write(img)


In [166]:
save_img('메인',1,'청바지',df.loc[5,'link'])

In [175]:
# word image 100개 검색한 데이터는 csv파일로, 이미지는 ch14_image폴더에 저장되는 함수   
def get_naver_save_image(word):
    import requests
    import pandas as pd
    from dotenv import load_dotenv
    import os
    load_dotenv() 
    client_id = os.getenv('Client_ID')
    client_secret = os.getenv('Client_Secret')
    
    url =f'https://openapi.naver.com/v1/search/image?query={word}&display=100'
    headers = {'X-Naver-Client-Id' : client_id,
               'X-Naver-Client-Secret' : client_secret}
    response = requests.get(url, headers=headers)
    
    items = response.json()['items']
    items_list  = [] #items_list초기화 
    for idx, item in enumerate(items):
        title = item.get('title')
        link = item.get('link')
        thumbnail = item.get('thumbnail')
        sizeheight = item.get('sizeheight')
        sizewidth = item.get('sizewidth')
        items_list.append({
            'no': idx+1,
            'title': title,
            'link' : link,
            'thumbnail' : thumbnail,
            'sizeheight':sizeheight,
            'sizewidth' :sizewidth
        })
        #link와 thumbnail을 저장 
        save_img('메인',idx+1,word,link)
        save_img('썸네일',idx+1,word,thumbnail)
        #20% ,40%, 60%,80% 지점 messae 출력
        if idx/20 == round(idx/20): #반올림
            print(f'= = = = {idx}% 진행중 입니다. = = = ')
    df = pd.DataFrame(items_list)
    df.to_csv(f'ch14_image/{word}.csv', index=False)
    print('~~~~~~이미지 및 사진 저장 완료')

In [174]:
get_naver_save_image('청바지')

= = = = 0% 진행중 입니다. = = = 
= = = = 20% 진행중 입니다. = = = 
= = = = 40% 진행중 입니다. = = = 
= = = = 60% 진행중 입니다. = = = 
= = = = 80% 진행중 입니다. = = = 
~~~~~~이미지 및 사진 저장 완료


## 2.3 XML  파일
### 1)기상예측 RSS 활용 
- http://www.kma.go.kr/repositary/xml/fct/mon/img/fct_mon1rss_108_20250529.xml

In [ ]:
# 방법2
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd


In [275]:
url = 'http://www.kma.go.kr/repositary/xml/fct/mon/img/fct_mon1rss_108_20250529.xml'
response = urlopen(url)
# response= status()  # response.getcode() 페이지 확인 하기
soup = BeautifulSoup(response,'xml') #
locals = soup.find_all('local_ta','')
# locals[1]
items_list = []
for local in locals:
    local_ta_name = local.select_one('local_ta_name').text
    week1_normalyear = local.select_one('week1_local_ta_normalYear').text #평년기온
    week1_similarRange = local.select_one('week1_local_ta_similarRange').text #기온범위
    week1_min = local.select_one('week1_local_ta_minVal').text
    week1_max = local.select_one('week1_local_ta_maxVal').text
    week2_normalyear = local.select_one('week2_local_ta_normalYear').text #평년기온
    week2_similarRange = local.select_one('week2_local_ta_similarRange').text #기온범위
    week2_min = local.select_one('week2_local_ta_minVal').text
    week2_max = local.select_one('week2_local_ta_maxVal').text
    items_list.append({
        '지역':local_ta_name,
        '1주평년기온':float(week1_normalyear),
        '1주범위':week1_similarRange ,
        '1주최저':week1_min ,
        '1주최고':week1_max ,
        '2주평년기온':week2_normalyear,
        '2주범위':week2_similarRange ,
        '2주최저':week2_min ,
        '2주최고':week2_max ,
    })
df = pd.DataFrame(items_list)
df

,지역,1주평년기온,1주범위,1주최저,1주최고,2주평년기온,2주범위,2주최저,2주최고
0,"전국(제주도,북한제외)",21.1,20.6~21.6,10,60,21.9,21.5~22.3,20,40
1,서울ㆍ인천ㆍ경기도,21.6,21.1~22.1,10,60,22.4,22.0~22.8,20,30
2,강원도 영서,20.3,19.7~20.9,10,60,21.3,20.7~21.9,20,30
3,강원도 영동,19.1,18.2~20.0,10,60,20.1,19.4~20.8,20,30
4,대전ㆍ세종ㆍ충청남도,21.5,21.0~22.0,10,60,22.3,21.9~22.7,20,40
5,충청북도,21.3,20.7~21.9,10,60,22.2,21.8~22.6,20,40
6,광주ㆍ전라남도,21.3,20.9~21.7,10,60,22.0,21.7~22.3,20,40
7,전북자치도,21.3,20.9~21.7,10,60,22.1,21.7~22.5,20,40
8,부산ㆍ울산ㆍ경상남도,21.3,20.8~21.8,10,60,22.1,21.7~22.5,20,40
9,대구ㆍ경상북도,20.9,20.3~21.5,10,60,21.9,21.3~22.5,20,40


### 2) 버스위치정보 조회(OpenAPI활용)
- data.go.kr에서 
    - *서울특별시_노선정보조회 서비스(busRouteId, stationNm-정류소명, station-정류소 id)
    - *서울특별시_버스위치정보조회 서비스(busRouteId ->버스들위치)

In [ ]:
# step1 :버스번호 -> busRouteId
# 노선정보조회 서비스  3번 getBusRouteList서비스

In [10]:
# key불러오기
from dotenv import load_dotenv
import os
load_dotenv() #dotenv_path='.env' 기본값
# print('key :', os.getenv('key'))
# print('skey :', os.getenv('skey'))

True

In [27]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlretrieve #url경로파일을 내pc에 저장
# from urllib.parse import quote #한글을 바꿀때
# busNum = '마포09'
busNum = '7011'
key = os.getenv('key') #일반인증키(Encoding)

url1 =f'http://ws.bus.go.kr/api/rest/busRouteInfo/getBusRouteList?ServiceKey={key}&strSrch={busNum}'
# print(url1)
# 방법1
# response = requests.get(url)
# soup = BeautifulSoup(response.text,'xml')

# 파일 저장
savefilename = 'data/ch14_busInfo.xml'
urlretrieve(url, savefilename) #url1 경로 파일을 savefilename으로 저장
xml = open(savefilename,'r', encoding='utf-8').read()
soup = BeautifulSoup(xml, 'xml')

#내가 원하는 버스 정보 확인
# soup.select('itemList') #배열
for item in soup.select('itemList'): #배열이라서 바로 for문 돌림
    busRouteNm = item.select_one('busRouteNm').text
    if busRouteNm == busNum: #버스라우트넘이랑 내가 찾는 버스랑 같은지
        busRouteId = item.select_one('busRouteId').text
        break
print('busRouteId =', busRouteId) 

busRouteId = 100100450


In [52]:
# step2 : busRouteId로 버스 정류장 목록(정류장 이름과 정류장 ID)
# 노선정보조회 서비스  4번 getStaionsByRouteList
import pandas as pd
url2 =f'http://ws.bus.go.kr/api/rest/busRouteInfo/getStaionByRoute?ServiceKey={key}&busRouteId={busRouteId}'
print(url2)
response = requests.get(url2)
soup = BeautifulSoup(response.text, 'xml')
itemLists = soup.find_all('itemList')
print(busNum + '번 정류장 갯수 :', len(itemLists))
bus_station = []
for itemList in itemLists:
    # 정류장이름, 정류장id, 경도, 위도
    stationNm = itemList.find('stationNm').text
    station   = itemList.find('station').text
    gpsX      = itemList.find('gpsX').text
    gpsY      = itemList.find('gpsY').text
    bus_station.append([stationNm,station,gpsX,gpsY])
df = pd.DataFrame(bus_station, columns=['정류소','id','경도','위도'])
df.head()

http://ws.bus.go.kr/api/rest/busRouteInfo/getStaionByRoute?ServiceKey=ZS3iEVZ82iG9dvZc3urYq03YK9x0dwPWsaXcaNd1XACv8yWys1TONILRRSHiRNyrFo1qVk379RdHIUhAtRaL5w%3D%3D&busRouteId=100100450
7011번 정류장 갯수 : 79


,정류소,id,경도,위도
0,은평공영차고지,111000907,126.88381,37.591802
1,덕은교.은평차고지앞,111000225,126.8834469451,37.589961128
2,DMC첨단산업센터,113000022,126.884787,37.585598
3,월드컵파크7단지,113000181,126.878963,37.581876
4,서부면허시험장,113000020,126.8802386693,37.5774012358


In [48]:
# Step3 : busRouteId로 버스 위치 정보 조회 2번 getBusPosByRtidList
url3 = f'http://ws.bus.go.kr/api/rest/buspos/getBusPosByRtid?ServiceKey={key}&busRouteId={busRouteId}'
print(url3)
response = requests.get(url3)
soup = BeautifulSoup(response.text,'xml')
bus_position = []
itemLists = soup.select('itemList')
# print('운행중인 버스 수 :', len(itemLists))
for itemList in soup.select('itemList'):  #soup.select('itemList') 운행중인 버스대수
    #차량번호, 혼잡도, 경도, 위도, 최종정류장id, 다음정류장id, 도착소요시간
    plainNo   = itemList.select_one('plainNo').text
    congetion = itemList.select_one('congetion').text
    congetion = '여유' if congetion == '3'\
                else '보통' if congetion =='4'\
                else'혼잡' if congetion == '5'\
                else '매우혼잡'
    gpsX      = itemList.select_one('gpsX').text
    gpsY      = itemList.select_one('gpsY').text
    lastStnId = itemList.select_one('lastStnId').text
    nextStId  = itemList.select_one('nextStId').text
    nextStTm  = itemList.select_one('nextStTm').text
    bus_position.append({
        '차량번호' : plainNo,
        '혼잡도' : congetion,
        '경도' : gpsX,
        '위도' : gpsY,
        '최종정류장ID' : lastStnId,
        '다음정류장ID' : nextStId,
        '도착소요시간' :nextStTm})
df_pos = pd.DataFrame(bus_position)
df_pos.head()

http://ws.bus.go.kr/api/rest/buspos/getBusPosByRtid?ServiceKey=ZS3iEVZ82iG9dvZc3urYq03YK9x0dwPWsaXcaNd1XACv8yWys1TONILRRSHiRNyrFo1qVk379RdHIUhAtRaL5w%3D%3D&busRouteId=100100450


,차량번호,혼잡도,경도,위도,최종정류장ID,다음정류장ID,도착소요시간
0,서울74사7256,여유,126.884245,37.584902,113000022,113000018,513
1,서울70사7794,여유,126.892971,37.568073,113000410,113000154,923
2,서울70사7798,여유,126.912502,37.551925,113000155,113000154,70
3,서울74사7206,여유,126.920839,37.547754,113000032,101000005,1400
4,서울74사7222,여유,126.923048,37.549226,113000139,101000005,1196


In [63]:
# id가 113000022인 정류소명
df.loc[df['id']== '113000022','정류소']. iloc[0] 
# df.loc[df['id']== '113000018','정류소']. iloc[0] #현재는 타입은 시리즈 -> 스칼라데이터로 변환 

'DMC첨단산업센터'

In [67]:
def station_name(row):
    row['최종정류소명'] = df.loc[df['id'] == row['최종정류장ID'], '정류소'].iloc[0]
    row['다음정류소명'] = df.loc[df['id'] == row['다음정류장ID'], '정류소'].iloc[0]
    return row

In [68]:
station_name(df_pos.loc[0])

차량번호            서울74사7256
혼잡도                    여유
경도             126.884245
위도              37.584902
최종정류장ID         113000022
다음정류장ID         113000018
도착소요시간                513
최종정류소명          DMC첨단산업센터
다음정류소명     월드컵파크3단지.난지천공원
Name: 0, dtype: object

In [69]:
df_pos.loc[0]

차량번호        서울74사7256
혼잡도                여유
경도         126.884245
위도          37.584902
최종정류장ID     113000022
다음정류장ID     113000018
도착소요시간            513
Name: 0, dtype: object

In [71]:
df_pos = df_pos.apply(station_name, axis=1)
df_pos

,차량번호,혼잡도,경도,위도,최종정류장ID,다음정류장ID,도착소요시간,최종정류소명,다음정류소명
0,서울74사7256,여유,126.884245,37.584902,113000022,113000018,513,DMC첨단산업센터,월드컵파크3단지.난지천공원
1,서울70사7794,여유,126.892971,37.568073,113000410,113000154,923,월드컵공원입구.문화비축기지입구,마포한강푸르지오
2,서울70사7798,여유,126.912502,37.551925,113000155,113000154,70,성산초등학교입구,마포한강푸르지오
3,서울74사7206,여유,126.920839,37.547754,113000032,101000005,1400,서울화력발전소입구,서울역버스환승센터
4,서울74사7222,여유,126.923048,37.549226,113000139,101000005,1196,극동방송,서울역버스환승센터
5,서울70사7795,여유,126.971356,37.559574,101000016,101000005,198,한국경제신문사.서소문역사공원,서울역버스환승센터
6,서울70사7789,여유,126.983281,37.560572,101000047,112000054,2668,남대문시장.회현역,충정로역2호선
7,서울74사7242,여유,126.990465,37.561101,101000052,112000054,1363,충무로역.한옥마을.한국의집,충정로역2호선
8,서울74사7246,여유,126.974814,37.55695,101000046,112000054,1086,남대문시장.회현역,충정로역2호선
9,서울70사7802,여유,126.956162,37.55741,113000478,113000475,132,아현역,웨딩타운


In [75]:
#컬럼명에 id가 포함된 컬럼 제거하기
drop_true = df_pos.columns.str.contains('ID')
drop_column_name = df_pos.columns[drop_true]
df_pos.drop(drop_column_name,axis=1)
# df_pos.drop(['최종정류장ID','다음정류장ID'],axis=1)

,차량번호,혼잡도,경도,위도,도착소요시간,최종정류소명,다음정류소명
0,서울74사7256,여유,126.884245,37.584902,513,DMC첨단산업센터,월드컵파크3단지.난지천공원
1,서울70사7794,여유,126.892971,37.568073,923,월드컵공원입구.문화비축기지입구,마포한강푸르지오
2,서울70사7798,여유,126.912502,37.551925,70,성산초등학교입구,마포한강푸르지오
3,서울74사7206,여유,126.920839,37.547754,1400,서울화력발전소입구,서울역버스환승센터
4,서울74사7222,여유,126.923048,37.549226,1196,극동방송,서울역버스환승센터
5,서울70사7795,여유,126.971356,37.559574,198,한국경제신문사.서소문역사공원,서울역버스환승센터
6,서울70사7789,여유,126.983281,37.560572,2668,남대문시장.회현역,충정로역2호선
7,서울74사7242,여유,126.990465,37.561101,1363,충무로역.한옥마을.한국의집,충정로역2호선
8,서울74사7246,여유,126.974814,37.55695,1086,남대문시장.회현역,충정로역2호선
9,서울70사7802,여유,126.956162,37.55741,132,아현역,웨딩타운


# qize yes24 사이트
- yes24 베스트셀러 정보를 ch14_yes24.csv(ch14_yes24.txt)로 출력 
- 순위, 책이름, 저자, 출판사, 가격 (1~48위) => 리스트 => 데이터프레임 => csv

- http://www.yes24.com/24/category/bestseller
- 수집 항목 <순위, 책이름, 저자및 출판사, 가격>
- 제출파일 : 1_유성미소스.ipynb/  1_소스및 실행결과.html / 1_yes24.csv


In [78]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

In [81]:
# 방법1
pages = 2
bestseller_list = [] #담을 빈리스트
with open('data/ch14_yes24.txt', 'w', encoding='utf-8')as f:
    pass
for page in range(1, pages+1):
    url = f'https://www.yes24.com/product/category/bestseller?pageNumber={page}'
    response = requests.get(url)
#     print(url) #주소 체크
#     print(response.status_code) #잘 연결되었는지 체크
    soup = BeautifulSoup(response.text, 'html.parser')
    rank_els = 


200
200


In [280]:
#1.웹 크롤링
from urllib.request import urlopen, Request
pages = 2
items_list = [] #초기화
for page in range(1, pages+1):
    url = f'https://www.yes24.com/product/category/bestseller?categoryNumber=001&pageNumber={page}'
    #print(url)
    response = urlopen(url)
    # print(response.status) #상태 코드 (정상상태 확인)
    soup = BeautifulSoup(response,
                        'html.parser')
    # print(soup)

    for item in items:
        rank_els = soup.select('div.img_upper > em.ico.rank')
        rank = [int(rank_el.text) for rank_el in rank_els]
        title_els = soup.select('div.item_info > div.info_row.info_name > a.gd_name')
        title = [(title_el.text) for title_el in title_els]
        art_els = soup.select('div.info_row.info_pubGrp > span > a')
        art = [(art_el.text) for art_el in art_els]
        price_els = soup.select('div.info_row.info_price > strong > em.yes_b')
        price = [(price_el.text) for price_el in price_els]
        items_list.append({
            '순위' :rank,
            '책제목' : title,
            '저자및출판사' : art,
            '가격' : price})

print(items_list)

    
    
#     print(len(rank), len(title), len(art) , len(price))#같지 않음 


#2.csv 파일 저장



[{'순위': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24], '책제목': ['청춘의 독서', '엄마의 말 연습', '단 한 번의 삶', '어른의 품격을 채우는 100일 필사 노트', '이로운 보수 의로운 진보', '소년이 온다', '초역 부처의 말', '어른의 행복은 조용하다', '모순', '듀얼 브레인', '쇼펜하우어 인생수업 : 한 번뿐인 삶 이렇게 살아라 (리커버 에디션)', '빛과 실', '에그박사 15', '혼모노', '위버멘쉬', '친구 사이에도 예의가 필요해', '급류', '괴수 8호 15 트리플특장판', '첫 여름, 완주', '친구가 상처 줄 때 똑똑하게 나를 지키는 법', '결국 국민이 합니다', '워런 버핏 웨이', '서랍에 저녁을 넣어 두었다', '2025 큰별쌤 최태성의 별별한국사 한국사능력검정시험 심화(1,2,3급) 상'], '저자및출판사': ['유시민', '웅진지식하우스', '윤지영', '카시오페아', '김영하', '복복서가', '김종원', '청림Life', '최강욱', '최강혁', '한겨레출판', '한강', '창비', '코이케 류노스케', '박재현', '포레스트북스', '태수', '페이지2북스', '양귀자', '쓰다', '이선 몰릭', '신동숙', '상상스퀘어', '쇼펜하우어', '김지민', '하이스트', '한강', '문학과지성사', '에그박사', '박송이', '홍종현', '주세종', '미래엔아이세움', '성해나', '창비', '프리드리히 니체', '떠오름', '한진아', '지니 킴', '박혜림', '길벗스쿨', '정대건', '민음사', '마츠모토 나오야', '유유리', '서울미디어코믹스(서울문화사)', '김금희', '무제', '이현아', '서영', '한빛에듀', '이재명', '오마이북', '로버트 해그스트롬', '신용우', '상상스퀘어', '한강', '문학과지성사', '최태성', '이투스북 '], '가격': ['17,01